Obtain satellite image after landslides in Global Landslide Catalog (see below site)
for Sentinel-2 and Landsat-8
https://catalog.data.gov/dataset/global-landslide-catalog-export
by Hiromu Daimaru 06 Aug. 2020<br>
Cloud masking argorithm was improved by Katsuto Shimizu (see below site)<br>
https://developers.google.com/earth-engine/api_docs <br>
This script requires geemap (see below site)<br>
https://github.com/giswqs/earthengine-py-notebooks

In [1]:
import geemap as emap
import ee
import datetime
import pandas as pd
import numpy as np
ee.Initialize()

In [2]:
# Function for cloud masking Sentinel-2 image
def maskS2clouds(image):
    qa = image.select('QA60')
    #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    #Both flags should be set to zero, indicating clear conditions.

    #Both flags should be set to zero, indicating clear conditions.
    #var mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    #Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# Cloud masking function by using cdi
# https://developers.google.com/earth-engine/api_docs
def s2_mask_cdi(img):
    cdi = ee.Algorithms.Sentinel2.CDI(img).gt(-0.5)
    return img.updateMask(cdi)

In [3]:
# Landsat 8 TOAのための雲マスク関数
def maskL8toa(image):
    qa = image.select('BQA')
    #Check that the cloud bit is off.
    #See https://landsat.usgs.gov/collectionqualityband
    mask = qa.bitwiseAnd(1 << 4).eq(0)
    return image.updateMask(mask)

In [4]:
#Landsat 4, 5, 7 surface reflectance 用のQA band による雲マスク関数
def cloudMaskL457(image):
  qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloud = qa.bitwiseAnd(1 << 5) \
          .And(qa.bitwiseAnd(1 << 7)) \
          .Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
  mask2 = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask2)

In [5]:
#Sentinel-2 用のNDVI算出関数
def ndviSn2(image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']))

In [6]:
#Landsat 8 用のNDVI算出関数
def ndviLs8(image):
    return image.addBands(image.normalizedDifference(['B5', 'B4']))

In [7]:
# This function gets NDVI from Landsat 5 imagery.
def ndviLs457(image):
    return image.normalizedDifference(['B4', 'B3'])


In [8]:
def getND(x, y, startDate, endDate, satellite, cloudness):
    # 対象地点のジオメトリを生成
    point = ee.Geometry.Point([x, y])
    #開始日の日付を'YYYY-MM-DD'形式で指定する
    startDay = ee.Date(startDate)
    #終了日の日付を'YYYY-MM-DD'形式で指定する
    endDay = ee.Date(endDate)
    if satellite == 'sn2':
        collection = ee.ImageCollection('COPERNICUS/S2').filterDate(startDay, endDay).filterBounds(point)\
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloudness)).map(s2_mask_cdi)
        medianImage = collection.median()
        ndImage = medianImage.addBands(medianImage.normalizedDifference(['B8', 'B4']))
        return ndImage
    elif satellite == 'ls8':
        collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate(startDay, endDay).filterBounds(point)\
        .filter(ee.Filter.lt('CLOUD_COVER', cloudness)).map(maskL8toa)
        medianImage = collection.median()
        ndImage = medianImage.addBands(medianImage.normalizedDifference(['B5', 'B4']))
        return ndImage

    elif satellite == 'ls7':
        collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate(startDay, endDay).filterBounds(point)\
        .filter(ee.Filter.lt('CLOUD_COVER_LAND', cloudness)).map(cloudMaskL457)
        medianImage = collection.median()
        ndImage = medianImage.addBands(medianImage.normalizedDifference(['B4', 'B3']))
        return ndImage

    elif satellite == 'ls5':
        collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate(startDay, endDay).filterBounds(point)\
        .filter(ee.Filter.lt('CLOUD_COVER_LAND', cloudness)).map(cloudMaskL457)
        medianImage = collection.median()
        ndImage = medianImage.addBands(medianImage.normalizedDifference(['B4', 'B3']))
        return ndImage
    
    else:
        return 0

In [9]:
# Set the dimention of study area
x_size = 0.5 # size of roi in decimal degree
y_size = 0.5 # size of roi in decimal degree

In [10]:
# Set the term of search period
af_term = 120
bf_term = 120
# Set the cloudness. Data below this value will be selected.
af_cloudness = 30
bf_cloudness = 30

In [11]:
# data csv file was dlownloaded from above site and was renamed
lsdf=pd.read_csv("dbase_nasa/GLC20200719.csv")

In [12]:
lsdf['date']=pd.to_datetime(lsdf['event_date'])

In [13]:
# Specify event record
#id_roi = '10775'
#id_roi = '10769'
id_roi = '9734'
#id_roi = '5463'

# variable become avairable by using @
df_roi = lsdf.query('event_id==@id_roi')

In [14]:
# Print the event record
df_roi.head()

,source_name,source_link,event_id,event_date,event_time,event_title,event_description,location_description,location_accuracy,landslide_category,...,admin_division_name,admin_division_population,gazeteer_closest_point,gazeteer_distance,submitted_date,created_date,last_edited_date,longitude,latitude,date
252,LA Times,http://www.latimes.com/local/lanow/la-me-ln-bi...,9734,05/20/2017 01:34:00 PM,NaN,Big Sur Landslide,Massive section of hillside on Big Sur at Mud ...,"Mud Creek, on Highway 1",exact,landslide,...,NaN,NaN,NaN,NaN,06/16/2017 01:34:00 PM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,-121.432384,35.865628,2017-05-20 13:34:00


In [15]:
eventDate = df_roi['date']

In [16]:
eventDate

252   2017-05-20 13:34:00
Name: date, dtype: datetime64[ns]

In [17]:
ts = (eventDate - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
eventDateDt = datetime.datetime.utcfromtimestamp(ts)

C:\Users\hirom\Anaconda3\envs\GEE\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  """Entry point for launching an IPython kernel.


In [18]:
eventDateDt

datetime.datetime(2017, 5, 20, 13, 34)

In [19]:
eventDateDt.strftime('%Y-%m-%d')

'2017-05-20'

In [20]:
# 指定した日付に有効な衛星のコードを返す関数
def SatFromDate(date):
    sn2Start = datetime.datetime(2015, 6, 23)
    ls8Start = datetime.datetime(2013, 4, 1)
    ls7Start = datetime.datetime(1999, 1, 1)
    ls5Start = datetime.datetime(1984, 5, 1)
    if date > sn2Start:
        return 'sn2'
    elif date > ls8Start:
            return 'ls8'
    elif date > ls7Start:
            return 'ls7'
    elif date > ls5Start:
            return 'ls5'
    else:
            return 0

In [21]:
stringDate = eventDateDt.strftime('%Y-%m-%d')

In [22]:
eventDateDt

datetime.datetime(2017, 5, 20, 13, 34)

In [23]:
af_startDate = eventDateDt + datetime.timedelta(days=1)
af_startDateString = af_startDate.strftime('%Y-%m-%d')
af_endDate = af_startDate + datetime.timedelta(days=af_term)
af_endDateString = af_endDate.strftime('%Y-%m-%d')

bf_startDate = af_startDate - datetime.timedelta(days=365)
bf_startDateString = bf_startDate.strftime('%Y-%m-%d')

bf_endDate = bf_startDate + datetime.timedelta(days=bf_term)
bf_endDateString = bf_endDate.strftime('%Y-%m-%d')

af_sat = SatFromDate(datetime.datetime.strptime(af_startDateString, '%Y-%m-%d') )
bf_sat = SatFromDate(datetime.datetime.strptime(bf_startDateString, '%Y-%m-%d') )

In [24]:
x_roi = df_roi['longitude']
y_roi = df_roi['latitude']
x_roi=float(x_roi)
y_roi=float(y_roi)

In [25]:
x_min = x_roi - x_size * 0.5
x_max = x_roi + x_size * 0.5
y_min = y_roi - y_size * 0.5
y_max = y_roi + y_size * 0.5

In [26]:
af_Image=getND(x_roi, y_roi, af_startDateString, af_endDateString, af_sat, af_cloudness)
bf_Image=getND(x_roi, y_roi, bf_startDateString, bf_endDateString, bf_sat, bf_cloudness)

In [27]:
af_nd = af_Image.select('nd')
bf_nd = bf_Image.select('nd')
nd_def = af_nd.subtract(bf_nd)

In [28]:
alos_chiri = ee.Image("CSP/ERGo/1_0/Global/ALOS_CHILI").select('constant')
alosChiliVis = {min: 0.0, max: 255.0}

#alosMtpi = ee.Image("CSP/ERGo/1_0/Global/ALOS_mTPI").select('AVE')
#alosMtpiVis = {min: -200.0, max: 200.0, 'palette': ['0b1eff', '4be450', 'fffca4', 'ffa011', 'ff0000'],}

In [29]:
# Set the study area
roi = ee.Geometry.Rectangle([x_min, y_min, x_max, y_max])

In [30]:
Map = emap.Map(center=[y_roi, x_roi], zoom=13)
Map.add_basemap('ROADMAP') # Add Google Map
Map.add_basemap('SATELLITE') # Add Satellite image

layer already on map: TileLayer(attribution='Google', name='Google Maps', options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms'], url='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}')
Basemap can only be one of the following:
  ROADMAP
  SATELLITE
  TERRAIN
  HYBRID
  ESRI
  Esri Ocean
  Esri Satellite
  Esri Standard
  Esri Terrain
  Esri Transportation
  Esri Topo World
  Esri National Geographic
  Esri Shaded Relief
  Esri Physical Map
  FWS NWI Wetlands
  FWS NWI Wetlands Raster
  Google Maps
  Google Satellite
  Google Terrain
  Google Satellite Hybrid
  NLCD 2016 CONUS Land Cover
  NLCD 2013 CONUS Land Cover
  NLCD 2011 CONUS Land Cover
  NLCD 2008 CONUS Land Cover
  NLCD 2006 CONUS Land Cover
  NLCD 2004 CONUS Land Cover
  NLCD 2001 CONUS Land Cover
  USGS NAIP Imagery
  USGS Hydrography
  USGS 3DEP Elevation
  OpenStreetMap.Mapnik
  OpenStreetMap.BlackAndWhite
  OpenStreetMap.DE
  OpenStreetMap.F

In [31]:
af_title = af_startDateString + '-' + af_endDateString + ' (' + af_sat + ')'
bf_title = bf_startDateString + '-' + bf_endDateString + ' (' + bf_sat + ')'

In [32]:
bf_nd

In [33]:
def truecolorVis(satellite):
    if satellite == 'sn2':
        return {'min': 500, 'max': 2000, 'gamma': 2.5, 'bands': ['B4', 'B3', 'B2']}
    elif satellite == 'ls8':
        return {'min': 0.0, 'max': 0.3, 'gamma': 2.5, 'bands': ['B4', 'B3', 'B2']}
    elif satellite == 'ls7':
        return {'min': 0, 'max': 3000, 'gamma': 1.4, 'bands': ['B3', 'B2', 'B1']}
    elif satellite == 'ls5':
        return {'min': 0, 'max': 3000, 'gamma': 1.4, 'bands': ['B3', 'B2', 'B1']}
    else:
        return {'min': 0.0, 'max': 0.3, 'bands': ['B3', 'B2', 'B1']}


In [34]:
#rgbVis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}
ndDefParams ={'min': -0.5, 'max': 0.5, 'palette': ['FFFF00','FF0000','FFFFFF', '00FFFF','0000FF']}

In [35]:
Map.addLayer(alos_chiri, alosChiliVis, 'ALOS CHILI')
#Map.addLayer(alosMtpi, alosMtpiVis, 'ALOS mTPI');
Map.addLayer(bf_Image, truecolorVis(bf_sat), bf_title)
Map.addLayer(af_Image, truecolorVis(af_sat), af_title)
Map.addLayer(nd_def, ndDefParams, 'NDVI change')

In [36]:
Map

Map(center=[35.86562803, -121.4323838], controls=(WidgetControl(options=['position'], widget=HBox(children=(To…